In [3]:
import os
import pandas as pd
from src import _PATH_DATA

In [6]:
DATASET_PATH = os.path.join(_PATH_DATA,"raw/Bugs/bugnist_256")

folders = [name for name in os.listdir(DATASET_PATH) if os.path.isdir(os.path.join(DATASET_PATH, name))]
files = [f"{folder}/{file}" for folder in folders for file in os.listdir(f"{DATASET_PATH}/{folder}")]
# write files as csv
with open(f"{DATASET_PATH}/files.csv", "w") as f:
    f.write("files\n")
    f.write("\n".join(files))

In [15]:
CSV_PATH = "../data/raw/Bugs/bugnist_256/files.csv"
VALIDATION_PATH = "../data/raw/Bugs/bugnist_256/validation.csv"
TRAIN_PATH = "../data/raw/Bugs/bugnist_256/train.csv"
TEST_PATH = "../data/raw/Bugs/bugnist_256/test.csv"
SAMPLE_PERCENTAGE = 0.20

# SEED = 9000 + 4206969420
SEED = 8963514

# Avoid accidentally overwriting data
error: str = ""
if os.path.exists(VALIDATION_PATH):
    error = f"{VALIDATION_PATH} already exists. "
if os.path.exists(TRAIN_PATH):
    error += f"{TRAIN_PATH} already exists. "
    
if os.path.exists(TEST_PATH):
    error += f"{TEST_PATH} already exists. "

if not os.path.exists(CSV_PATH):
    error += f"CSV path {CSV_PATH} does not exist. "

if SAMPLE_PERCENTAGE < 0 or SAMPLE_PERCENTAGE > 1:
    error += f"Sample percentage {SAMPLE_PERCENTAGE} is not between 0 and 1. "

if error:
    raise ValueError(error.strip())

original_csv = pd.read_csv(CSV_PATH)

labels = original_csv["files"].map(lambda x: x[0:2])
grouped = original_csv.groupby(labels.values)
test_subset = grouped.sample(frac=SAMPLE_PERCENTAGE, random_state=SEED)

test_subset.to_csv(TEST_PATH, index=False)

difference = original_csv.index.difference(test_subset.index)
original_without_test_subset = original_csv.loc[difference]


labels = original_without_test_subset["files"].map(lambda x: x[0:2])
grouped = original_without_test_subset.groupby(labels.values)
val_subset = grouped.sample(frac=SAMPLE_PERCENTAGE, random_state=SEED)

val_subset.to_csv(VALIDATION_PATH, index=False)

difference = original_without_test_subset.index.difference(val_subset.index)
train_subset = original_without_test_subset.loc[difference]

train_subset.to_csv(TRAIN_PATH, index=False)